In [6]:
import pandas as pd

test_cases_df = pd.read_csv("/content/pronoun_testcases.csv")
display(test_cases_df)

,input_text,target_gender,expected_output
0,He is going to the market.,female,She is going to the market.
1,His book is on the table.,female,Her book is on the table.
2,I saw him yesterday.,female,I saw her yesterday.
3,He hurt himself.,female,She hurt herself.
4,I called him last night.,female,I called her last night.
5,That is his car.,female,That is her car.
6,He told me about his trip.,female,She told me about her trip.
7,The teacher gave him a warning.,female,The teacher gave her a warning.
8,He blames himself for the mistake.,female,She blames herself for the mistake.
9,He brought his laptop.,female,She brought her laptop.


In [7]:
import string
import spacy

nlp = spacy.load("en_core_web_sm")

def transform_pronouns_improved(input_text, target_gender):
    """
    Transforms gendered pronouns in a sentence from one gender to the opposite,
    with improved logic for capitalization, punctuation, and "her" ambiguity using spaCy for POS tagging.

    Args:
      input_text: The sentence to transform.
      target_gender: The target gender ('male' or 'female').

    Returns:
      The transformed sentence.
    """
    doc = nlp(input_text)
    result_words = []

    pronoun_map = {
        'male': {'he': 'she', 'him': 'her', 'his': 'her', 'himself': 'herself'},
        'female': {'she': 'he', 'her': 'him', 'herself': 'himself', 'her_poss': 'his'}
    }

    for token in doc:
        word = token.text
        lower_word = word.lower()
        is_capitalized = word.istitle()
        new_word = word

        if target_gender == 'female':
            if lower_word in pronoun_map['male']:
                replacement = pronoun_map['male'][lower_word]
                if is_capitalized:
                    replacement = replacement.capitalize()
                new_word = replacement

        elif target_gender == 'male':
            if lower_word in ['she', 'herself']:
                replacement = pronoun_map['female'][lower_word]
                if is_capitalized:
                    replacement = replacement.capitalize()
                new_word = replacement
            elif lower_word == 'her':

                if token.pos_ == 'DET':
                    replacement = pronoun_map['female']['her_poss']
                elif token.pos_ == 'PRON':
                     replacement = pronoun_map['female']['her']
                else:
                     replacement = pronoun_map['female']['her']

                if is_capitalized:
                    replacement = replacement.capitalize()
                new_word = replacement

        result_words.append(new_word)

    return " ".join(result_words)

In [8]:
results = []
for index, row in test_cases_df.iterrows():
    input_text = row['input_text']
    target_gender = row['target_gender']
    transformed_text = transform_pronouns_improved(input_text, target_gender)
    results.append({
        'input_text': input_text,
        'target_gender': target_gender,
        'expected_output': row['expected_output'],
        'transformed_output': transformed_text
    })

results_df = pd.DataFrame(results)
display(results_df)

,input_text,target_gender,expected_output,transformed_output
0,He is going to the market.,female,She is going to the market.,She is going to the market .
1,His book is on the table.,female,Her book is on the table.,Her book is on the table .
2,I saw him yesterday.,female,I saw her yesterday.,I saw her yesterday .
3,He hurt himself.,female,She hurt herself.,She hurt herself .
4,I called him last night.,female,I called her last night.,I called her last night .
5,That is his car.,female,That is her car.,That is her car .
6,He told me about his trip.,female,She told me about her trip.,She told me about her trip .
7,The teacher gave him a warning.,female,The teacher gave her a warning.,The teacher gave her a warning .
8,He blames himself for the mistake.,female,She blames herself for the mistake.,She blames herself for the mistake .
9,He brought his laptop.,female,She brought her laptop.,She brought her laptop .
